In [30]:
#
# Left alligned
import os 
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont
import tkinter as tk
import requests
from rich.pretty import pprint
import pandas as pd
import base64
import qrcode
import yaml 

base_info = yaml.safe_load(open("base-info.yml"))["base"]
pprint(base_info)

Address = base_info["Address"]
PostalCode = base_info["PostalCode"]
Country = base_info["Country"]


def encode_image(image_path):
    with open(image_path, "rb") as img_file:
        return base64.b64encode(img_file.read()).decode("utf-8")


def generate_vcard(contact):
    contact = contact.to_dict()
    vcard_lines = [
        "BEGIN:VCARD",
        "VERSION:3.0",
        f"N:{contact.get('Last Name', '')};{contact['First Name']};{contact.get('Middle Name', '') if contact.get('Middle Name', '') else ''}",
        f"FN:{contact['First Name']}",
        (
            f"NICKNAME: {contact.get('Nickname', '')}"
            if contact.get("Nickname", None)
            else ""
        ),
        (
            f"X-PHONETIC-FIRST-NAME:{contact.get('Thai First Name', '')}"
            if contact.get("Thai First Name", None)
            else ""
        ),
        (
            f"X-PHONETIC-LAST-NAME:{contact.get('Thai Last Name', '')}"
            if contact.get("Thai Last Name", None)
            else ""
        ),
        f"TITLE:{contact['Job Title']}",
        f"ORG: {base_info['Organization']}",
        f"EMAIL;type=INTERNET;type=WORK;type=pref:{contact['Email']}",
        f"TEL;type=CELL;type=VOICE;type=pref:{'+'+contact['Phone']}",
        f"ADR;type=WORK;type=pref:;;{Address};Bangkok;{PostalCode};{Country}",
        f"NOTE:{contact['Note']}" if contact.get("Note", None) else "",
        (
            f"IMPP;X-SERVICE-TYPE=Line:LineID:{contact.get('Line ID', '')}"
            if contact.get("Line ID", None)
            else ""
        ),
        (
            f"item1.URL:{contact.get('Website', '')}"
            if contact.get("Website", None)
            else ""
        ),
        "item1.X-ABLabel:Website" if contact.get("Website", None) else "",
        (
            f"item2.URL:{contact.get('LinkedIn URL', '')}"
            if contact.get("LinkedIn URL", None)
            else ""
        ),
        "item2.X-ABLabel:LinkedIn" if contact.get("LinkedIn URL", None) else "",
        "END:VCARD",
    ]

    vcard = "\n".join([line for line in vcard_lines if line])
    return vcard


def add_text_to_image(image, text, y_position, color, size, bold=False):
    draw = ImageDraw.Draw(image)
    font_path = "fonts/Inter-Bold.ttf" if bold else "fonts/Inter-Medium.ttf"
    font = ImageFont.truetype(font_path, size)
    text_bbox = draw.textbbox((0, 0), text, font=font)
    text_width = text_bbox[2] - text_bbox[0]
    image_width = image.size[0]
    x_position = (image_width - text_width) // 2  # Center align

    draw.text((x_position, y_position), text, fill=color, font=font, align="center")
    return image


def convert_phone_number(phone_number):
    if phone_number.startswith("66"):
        return "+66-{}-{}-{}".format(
            phone_number[2:4], phone_number[4:7], phone_number[7:]
        )
    elif phone_number.startswith("60"):
        return "+60-{}-{}-{}".format(
            phone_number[2:4], phone_number[4:7], phone_number[7:]
        )
    else:
        return "Invalid phone number"



printout = False
def main():
    orange = (232, 95, 38)  # sRGB Orange color
    blue = (48, 62, 122)  # Blue color
    size = 72
    offset = 20
    START = 1 # start at 1 
    END = 2
    
    df = pd.read_excel(base_info["ExcelFile"], dtype=str)
    df = df.applymap(lambda x: None if pd.isna(x) else x)

    template_image_path = base_info["Template"]  # Replace with your template image file
    template_image = Image.open(template_image_path)
    
    if os.path.exists("outputcard") == False:
        os.mkdir("outputcard")

    for index, contact in df.iterrows():
        print(contact)
        # print(index)
        if index  + 1 >= START:
            vcard = generate_vcard(contact)
            if printout:
                print(f"{vcard}")

            qr = qrcode.QRCode(
                version=4,
                error_correction=qrcode.constants.ERROR_CORRECT_L,
                box_size=10,
                border=4,
            )
            qr.add_data(vcard)
            qr.make(fit=True)

            qr_img = qr.make_image(fill_color=orange, back_color="white")

            qr_size = (1000, 1000)  # Adjust size as needed
            qr_img = qr_img.resize(qr_size)

            business_card = template_image.copy()
            template_width, template_height = template_image.size
            qr_width, qr_height = qr_img.size
            qr_position = (
                (template_width - qr_width) // 2,
                int((template_height - qr_height) // 2 * 1.5),
            )
            contact_position = (
                (template_width - qr_width) // 2,
                int((template_height - qr_height) // 2 * 1.5 + qr_height + 50),
            )
            business_card.paste(qr_img, qr_position)

            Name = contact["First Name"].strip()
            if contact.get("Middle Name"):
                Name += f" {contact['Middle Name'].strip()}"
            Name += f" {contact['Last Name'].strip()}"

            add_text_to_image(
                business_card, Name, 500 + offset + size * 1 - 15, orange, size, bold=True
            )
            add_text_to_image(
                business_card,
                contact["Job Title"],
                500 + (offset + size) * 2,
                blue,
                size,
            )
            add_text_to_image(
                business_card,
                contact["Email"],
                500 + (offset + size) * 3,
                blue,
                size,
            )
            add_text_to_image(
                business_card,
                convert_phone_number(contact["Phone"]),
                500 + (offset + size) * 4,
                blue,
                size,
            )
            add_text_to_image(
                business_card,
                "Scan to save to Contacts",
                template_height // 2 + 850,
                blue,
                size,
                bold=True,
            )

            output_path = f"outputcard/{index+1}_E-Card_{Name.strip()}.png"  # Adjust file naming as needed
            business_card.save(output_path)
            if printout:
                plt.imshow(business_card)
                plt.axis("off")
                plt.show()
                print("\n\n\n\n\n")
            print(f"Business card saved as {output_path}")
        if index+ 1 <= END:
            pass
        else:
            break


if __name__ == "__main__":
    main()

{
│   'Address': '234 Soi Sukhumvit 101 (Punnawithi 44), Bangchak, Phrakanong',
│   'PostalCode': '10260',
│   'Country': 'Thailand',
│   'Organization': 'PCS Security and Facility Services Limited',
│   'ExcelFile': 'Application for Virtual Business Card  (Responses) (8).xlsx',
│   'Template': 'template.png'
}

/var/folders/kl/36jj5f7x65x2wm9_48xpty780000gn/T/ipykernel_7579/641131406.py:116: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: None if pd.isna(x) else x)


Timestamp                                        2024-07-03 09:57:10
First Name                                                   Tinapat
Last Name                                                    Limsila
Nickname                                                        Game
Job Title                                          IOT Lead Engineer
Thai First Name                                              ทินภัทร
Thai Last Name                                              ลิ่มศิลา
Email                                            tinapatli@pcs.co.th
Phone                                                    66982488442
LinkedIn URL                     https://www.linkedin.com/in/gametl/
Note               I work on PCS IoT Athena. we do custom IoT sol...
Line ID                                         mynameisgame44454445
Website                                   https://gametl.vercel.app/
Middle Name                                                     None
Name: 0, dtype: object
Business ca